2.2.1


In [9]:
import numpy as np
import pyslam

def run_slam(video_path: str, calib_path: str) -> np.ndarray:
    """
    Run pySLAM on a video and return a 3D point cloud as a NumPy array.
    (No visualization, no Open3D)
    """
    cfg = config.Config("config.yaml")

    # --- configure dataset ---
    cfg["DATASET"]["type"] = "VIDEO_DATASET"
    cfg["DATASET"]["VIDEO_DATASET"]["path"] = video_path
    cfg["DATASET"]["VIDEO_DATASET"]["camera_settings"] = calib_path

    # --- enable dense reconstruction ---
    cfg["GLOBAL_PARAMETERS"]["kUseVolumetricIntegration"] = True
    cfg["GLOBAL_PARAMETERS"]["kVolumetricIntegrationType"] = "TSDF"
    cfg["GLOBAL_PARAMETERS"]["kUseDepthEstimatorInFrontEnd"] = True

    # --- run SLAM ---
    slam_system = slam.SLAM(cfg)
    slam_system.run()

    # --- get point cloud (Nx3 numpy array) ---
    dense_integrator = slam_system.get_dense_integrator()
    points = dense_integrator.get_points_numpy()  # suppose it returns np.ndarray
    return points


def compute_scene_size(points: np.ndarray):
    """
    Compute simple bounding box dimensions (length, width, height)
    from a NumPy point cloud array of shape (N, 3).
    """
    if points is None or len(points) == 0:
        raise ValueError("No points reconstructed from SLAM.")

    # Remove NaNs or infs
    valid = np.isfinite(points).all(axis=1)
    points = points[valid]

    # Compute min/max per axis
    mins = np.min(points, axis=0)
    maxs = np.max(points, axis=0)

    dims = maxs - mins  # [dx, dy, dz]
    return dims, mins, maxs


def scale_to_real_world(dims, known_real_length=None, known_axis=0):
    """
    Optional: rescale dimensions to meters if one known real-world length is given.
    known_axis = 0 (x), 1 (y), or 2 (z)
    """
    if known_real_length is None:
        return dims, 1.0
    scale = known_real_length / dims[known_axis]
    return dims * scale, scale


def main():
    video_path = "input_scene.mp4"
    calib_path = "camera.yaml"  # your calibration file

    print("Running SLAM... (this can take a while)")
    points = run_slam(video_path, calib_path)

    print(f"Reconstructed {len(points)} 3D points.")
    dims, mins, maxs = compute_scene_size(points)

    # Optional: if you know a real object’s size to fix scale
    # e.g., wall width known = 3.2 meters
    dims_m, scale = scale_to_real_world(dims, known_real_length=3.2, known_axis=0)

    print("Bounding box (in SLAM units):", dims)
    print(f"Scale factor: {scale:.3f}")
    print("Estimated scene size (meters):")
    print(f"  Length (x): {dims_m[0]:.2f} m")
    print(f"  Width  (y): {dims_m[1]:.2f} m")
    print(f"  Height (z): {dims_m[2]:.2f} m")


if __name__ == "__main__":
    main()


Running SLAM... (this can take a while)


NameError: name 'config' is not defined

In [10]:
import open3d as o3d

# Load or capture point cloud
pcd = o3d.io.read_point_cloud("room_scan.ply")

# Visualize
o3d.visualization.draw_geometries([pcd])

# Measure distance between points
points = np.asarray(pcd.points)
dist = np.linalg.norm(points[0] - points[1])
print(f"Distance: {dist:.2f} m")

ModuleNotFoundError: No module named 'open3d'

In [11]:
!pip install open3d

ERROR: Could not find a version that satisfies the requirement open3d (from versions: none)
ERROR: No matching distribution found for open3d
